In [1]:
"""Environment configuration loader with interactive setup."""
## TODO:: Something about automagically making sure pip install happens if missing dependencies

import os
from pathlib import Path
from getpass import getpass
from typing import List, Optional
from dotenv import load_dotenv
import sys

# SNIPPETS (needs cleaning up)
# Add the 'src' directory to Python's search path
sys.path.append(os.path.abspath("src"))

# If your code is actually in the 'agents' folder, uncomment the line below instead:
sys.path.append(os.path.abspath("agents"))

class EnvironmentConfig:
    """Manages loading and validation of environment variables."""
    
    REQUIRED_VARS = [
        "OPENROUTER_API_KEY",
        "RUNPOD_API_KEY",
        "RUNPOD_ENDPOINT_ID"
    ]
    
    def __init__(self, env_path: Optional[Path] = None):
        """Initialize configuration loader.
        
        Args:
            env_path: Path to .env file. Defaults to ~/workspace/.env
        """
        self.env_path = env_path or Path.home() / "workspace" / ".env"
        self._load_environment()
    
    def _load_environment(self) -> None:
        """Load environment variables from .env file."""
        load_dotenv(self.env_path, override=True)
        print(f"Environment source: {self.env_path}\n")
    
    def _prompt_for_missing_var(self, var_name: str) -> None:
        """Prompt user for missing environment variable.
        
        Args:
            var_name: Name of the environment variable
        """
        print(f"WARNING: {var_name} not found in .env file.")
        print("Please review the README or .env.example for setup instructions.")
        
        user_input = getpass(f"Please enter value for {var_name}: ")
        
        if user_input.strip():
            os.environ[var_name] = user_input.strip()
        else:
            print(f"Skipping {var_name} (no input provided)...")
    
    def _display_var_status(self, var_name: str) -> None:
        """Display the status of an environment variable.
        
        Args:
            var_name: Name of the environment variable
        """
        value = os.getenv(var_name)
        
        if value:
            print(f"{var_name:<25}: {value[:5]}...")
        else:
            print(f"{var_name:<25}: MISSING")
    
    def validate_and_setup(self) -> None:
        """Validate required variables and prompt for missing ones."""
        for var in self.REQUIRED_VARS:
            if not os.getenv(var):
                self._prompt_for_missing_var(var)
            
            self._display_var_status(var)
        
        print("-" * 40)
        self._raise_if_missing()
    
    def _raise_if_missing(self) -> None:
        """Raise ValueError if any required variables are missing."""
        missing = [var for var in self.REQUIRED_VARS if not os.getenv(var)]
        
        if missing:
            raise ValueError(
                f"Critical configuration missing: {', '.join(missing)}"
            )
    
    @classmethod
    def setup(cls, env_path: Optional[Path] = None) -> "EnvironmentConfig":
        """Convenience method to load and validate configuration.
        
        Args:
            env_path: Path to .env file. Defaults to ~/workspace/.env
            
        Returns:
            Configured EnvironmentConfig instance
            
        Raises:
            ValueError: If required variables are missing after prompting
        """
        config = cls(env_path)
        config.validate_and_setup()
        return config


# Usage
if __name__ == "__main__":
    config = EnvironmentConfig.setup()

Environment source: /home/jovyan/workspace/.env

OPENROUTER_API_KEY       : sk-or...
RUNPOD_API_KEY           : rpa_6...
RUNPOD_ENDPOINT_ID       : a48mr...
----------------------------------------


In [2]:
import os
import google.auth

# 1. Disable Vertex AI to stop looking for Google Cloud credentials
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"

# 2. (Optional) Set dummy project values if code expects them, 
#    but we won't actually auth against them.
os.environ.setdefault("GOOGLE_CLOUD_PROJECT", "openrouter-project")
os.environ["GOOGLE_CLOUD_LOCATION"] = "global"

# --- Commented out to prevent DefaultCredentialsError ---
# _, project_id = google.auth.default()
# os.environ.setdefault("GOOGLE_CLOUD_PROJECT", project_id)
# --------------------------------------------------------

# Configuration dictionary
config = {
    "model": "google/gemini-2.5-flash-lite", # We will override this in the agent setup if needed
    "verbose": True
}

In [4]:
import sys
import os

# Add the 'agents' folder to the path so we can import 'blogger_agent' from inside it
sys.path.append("/home/jovyan/workspace/agents")
sys.path.append("/home/jovyan/workspace/agents/blog-writer")

import asyncio

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from blogger_agent.agent import root_agent
from google.genai import types as genai_types


async def main():
    """Runs the agent with a sample query."""
    session_service = InMemorySessionService()
    await session_service.create_session(
        app_name="app", user_id="test_user", session_id="test_session"
    )
    runner = Runner(
        agent=root_agent, app_name="app", session_service=session_service
    )

    queries = [
        "I want to write a blog post about the new features in the latest version of the ADK.",
        "looks good, write it",
        "1",
        "looks good, I approve",
        "yes",
        "my_new_blog_post.md",
    ]

    for query in queries:
        print(f">>> {query}")
        async for event in runner.run_async(
            
            user_id="test_user",
            session_id="test_session",
            new_message=genai_types.Content(
                role="user", 
                parts=[genai_types.Part.from_text(text=query)]
            ),
        ):
            if event.is_final_response() and event.content and event.content.parts:
                print(event.content.parts[0].text)


if __name__ == "__main__":
    await main()

>>> I want to write a blog post about the new features in the latest version of the ADK.


It appears there are several prominent "ADK" (Assessment and Deployment Kit, Agent Development Kit, Android Development Kit, etc.) across different platforms (Windows, Google, Android, AWS, Azure). To create a specific and accurate blog post outline, please clarify which ADK you are referring to.

In the meantime, here is a general blog post outline that can be tailored once the specific ADK is identified. This outline includes sections for new features, potential code snippets, and technical deep dives, as is common when discussing updates to a development kit.

---

# Title: Unveiling the Future: A Deep Dive into the Latest ADK Features (Please Specify Which ADK: Windows, Google Agent, Android, AWS, Azure, etc.)

## I. Introduction
*   **A. Hook:** Briefly discuss the rapid evolution of technology and the importance of updated development kits in keeping pace.
*   **B. Introducing the ADK (Placeholder for Specific ADK):**
    *   Explain what the ADK is in general terms and its role 